In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

from ese.analysis.analyze_inf import load_cal_inference_stats
# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml results_cfg 

log:
    root: '/storage/vbutoi/scratch/ESE/inference'
    inference_groups: 
        - '10_04_24_ISLES_3D_FineGrainTempSweep'

options:
    verify_graceful_exit: True
    equal_rows_per_cfg_assert: False 

In [ ]:
inference_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=True
)

In [4]:
# for ikey in inference_df:
#     print(ikey)

In [ ]:
inference_df['experiment_model_dir'].unique()

In [6]:
# Add necessary column and do a check to ensure that we only are dealing with one model.
def abs_vol_estimation_error(soft_volume, gt_volume):
    return np.abs(soft_volume - gt_volume)

inference_df.augment(abs_vol_estimation_error)
assert len(inference_df['experiment_model_dir'].unique()) == 1
exp_model_dir = inference_df['experiment_model_dir'].unique()[0]

# Gather the per-image optimal temperatures.

In [7]:
sub_cols_to_keep = [
    'abs_vol_estimation_error',
    'temperature',
    'data_id',
    'split'
]
# Filter out the columns we want to keep
individual_temp_df = inference_df[sub_cols_to_keep].drop_duplicates().reset_index(drop=True)
# Get the optimal temperature for each data_id
opt_temps_df = individual_temp_df.loc[individual_temp_df.groupby('data_id')['abs_vol_estimation_error'].idxmin()].reset_index(drop=True)

In [9]:
# We want to make a dictionary that maps from data_id to optimal temperature
data_id_to_opt_temp = dict(zip(opt_temps_df.data_id, opt_temps_df.temperature))

In [12]:
import json
# Now we are going to save this dictionary to a file in the exp_model_dir called 'opt_temps.'
opt_temp_file = os.path.join(exp_model_dir, 'opt_temps.json')
# Save the dictionary using json
with open(opt_temp_file, 'w') as f:
    json.dump(data_id_to_opt_temp, f)